# Лабораторная работа 4

## Нейронные сети

## Задание
1. Вам необходимо реализовать все объявленные методы в папке **mla**.
2. Вам необходимо реализовать класс NeuralNet используя методы из папки **mla** в ячейке 1.
3. Проверить работу на синтетических данных в ячейке 2.
4. Создать нейросеть и подобрать ее архитектуру и параметры для датасета из **вашего варианта** в ячейке 3.
5. Проверить работу и точность с помощью заранее объявленной строки в ячейке 3 - `assert roc_auc_score(y_test_onehot[:, 0], predictions[:, 0]) >= 0.8`


In [26]:
from lab_4.mla.neuralnet import NeuralNet

In [27]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_samples=750, n_features=10, n_informative=8, random_state=1111, n_classes=2, class_sep=2.5, n_redundant=0
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=1111)

## Проверка работы

In [28]:
from mla.basic import Dense, Activation, Dropout
from mla.parameters import Parameters
from mla.constraints import MaxNorm
from mla.regularizers import L2
from mla.optimizers import Adadelta, Adam
from mla.utils import one_hot

from sklearn.metrics import roc_auc_score

y_train_onehot = one_hot(y_train)
y_test_onehot = one_hot(y_test)

model = NeuralNet(
    layers=[
        Dense(256, Parameters(init_name="uniform", regularizers={"W": L2(0.05)})),
        Activation("relu"),
        Dropout(0.5),
        Dense(128, Parameters(init_name="normal", constraints={"W": MaxNorm()})),
        Activation("relu"),
        Dense(2),
        Activation("softmax"),
    ],
    loss_name="categorical_crossentropy",
    optimizer=Adadelta(),
    metric_name="accuracy",
    batch_size=64,
    max_epochs=25,
)
model.fit(X_train, y_train_onehot)
predictions = model.predict(X_test)
assert roc_auc_score(y_test_onehot[:, 0], predictions[:, 0]) >= 0.95

  0%|          | 0/25 [00:00<?, ?it/s]

In [29]:
roc_auc_score(y_test_onehot[:, 0], predictions[:, 0])

1.0

## Свой датасет

In [30]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('Diabetes Binary Classification.csv')
data

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [31]:
# Разделение данных на признаки и целевую переменную
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование целевой переменной в one-hot представление
y_train_onehot = one_hot(y_train)
y_test_onehot = one_hot(y_test)

In [32]:
# Создание модели
model = NeuralNet(
    layers=[
        Dense(128, Parameters(init_name="uniform", regularizers={"W": L2(0.01)})),
        Activation("leakyrelu"),
        Dropout(0.5),
        Dense(64, Parameters(init_name="normal", constraints={"W": MaxNorm()})),
        Activation("leakyrelu"),
        Dense(2),
        Activation("softmax"),
    ],
    loss_name="categorical_crossentropy",
    optimizer=Adam(0.1),
    metric_name="accuracy",
    batch_size=64,
    max_epochs=50,
    verbose=True
)

# Обучение модели
model.fit(X_train, y_train_onehot)

# Предсказания
predictions = model.predict(X_test)

# Проверка метрики AUC-ROC
roc_auc = roc_auc_score(y_test_onehot[:, 0], predictions[:, 0])
print(f"AUC-ROC: {roc_auc:.4f}")

  0%|          | 0/50 [00:00<?, ?it/s]

AUC-ROC: 0.8171


In [33]:
# Утверждение, что модель достигает порога качества
assert roc_auc >= 0.8, "Model did not reach the expected ROC-AUC threshold."